In [1]:
import mysql.connector
import pandas as pd
import bcrypt

def establish_sql_connection():
    # note: for testing purposes, run on localhost
    db = mysql.connector.connect(
        host = "localhost",
        port = 1001,
        user = "root",
        password = "dsa3101",
        database = "secdb"
        )
    cursor = db.cursor()
    # uncomment below for dockerised usage
    # db = mysql.connector.connect(
    #     host = "database",
    #     user = "root",
    #     password = "dsa3101",
    #     database = "secdb"
    #     )
    return db, cursor

In [2]:
# add users
def add_user_roles(filepath):

    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## read users from path and change to QUERY format
        file = open(filepath, 'r')
        values = file.read().split("\n")
        values = list(map(lambda x: f'(\'{x}\')',values))
        values = ", ".join(values)
        query = f'INSERT INTO User_roles(role) VALUES {values};'

        cursor.execute(query)
        db.commit()
        print(f"Number of rows affected: {cursor.rowcount}")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")


In [3]:
add_user_roles('User_roles.txt')

db established: True
Number of rows affected: 2


In [100]:
def add_user(username, password, role, email=None):
    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## generate hash and salt
        salt = bcrypt.gensalt()
        hash = bcrypt.hashpw(password.encode('utf-8'), salt)

        ## get role id
        query = f'SELECT id FROM User_roles \
                                 where role = \'{role}\';'
        cursor.execute(query)
        role_id=cursor.fetchall()[0][0]

        ## insert query
        query = f"\
            INSERT INTO Users\
            (email, username, role_id, salt, hash)\
            VALUES (\'{email}\', \'{username}\', {role_id}, \'{salt.decode('utf-8')}\', \'{hash.decode('utf-8')}\')"
        cursor.execute(query)
        db.commit()

    except mysql.connector.Error as err:
        print(f"Database error: {err}")

    finally:
        cursor.close()
        db.close()



In [101]:
add_user('sec1','sec1','security','sec1@nus.edu.sg')

db = establish_sql_connection()
cursor = db.cursor()
query = "SELECT * from Users"
cursor.execute(query)
Users = cursor.fetchall()
print(f'Users: {Users}')
cursor.close()
db.close()

db established: True
Users: [(1, 'sec1', 'sec1@nus.edu.sg', 1, '$2b$12$yefe8Q4bk54xKpBPItKva.', '$2b$12$yefe8Q4bk54xKpBPItKva.WBg5Z9bPcmii7SLmj8k0SO1jokAlpr.')]


In [109]:
def authenticate(user,password):
    try:
        db = establish_sql_connection()
        cursor = db.cursor()
        query = f"SELECT hash from Users where username = \'{user}\'"
        cursor.execute(query)
        hash_stored = cursor.fetchall()

        ## Check username
        if len(hash_stored) == 0:
            print("User not found")
            return False
        
        ## Pull shash salt
        hash_stored = hash_stored[0][0]
        query = f"SELECT salt from Users where username = \'{user}\'"
        cursor.execute(query)
        salt = cursor.fetchall()[0][0]
        hash_entered = bcrypt.hashpw(password.encode('utf-8'),salt.encode('utf-8'))
    
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    
    finally:
        cursor.close()
        db.close()

    ## Check results
    authenticated = bcrypt.checkpw(password.encode('utf-8'),hash_stored.encode('utf-8'))
    
    return authenticated




In [113]:
authenticate("sec1","sec1")
authenticate("sec1","sec2")
authenticate("sec2","sec2")

User not found


False

In [ ]:
data = pd.read_csv("back-end/data/data_except_priority&status.csv")
df = pd.DataFrame(data)


In [6]:
def add_incident_types(filepath):
    try: 
        ## establish connection
        db, cursor = establish_sql_connection()
        if db is None:
            print("add_incident_types | Failed to establish a database connection.")
            return
        print(f'add_incident_types | db established: {db.is_connected()}')

        ## read incident types from path
        with open(filepath, 'r') as file:
            lines = file.readlines()
            values = [tuple(line.strip().split(", ")) for line in lines]

        # Format values for the query
        formatted_values = ", ".join([f"('{type}', '{priority}')" for type, priority in values])

        query = f'INSERT INTO Incident_types(type, default_priority) VALUES {formatted_values};'

        cursor.execute(query)
        db.commit()
        print(f"add_incident_types | Incident types added: {cursor.rowcount}")

        cursor.close()
        db.close()

    except mysql.connector.Error as err:
        print(f"add_incident_types | database error: {err}")


In [22]:
add_incident_types('Incident_types.txt')

add_incident_types | db established: True
('LOST AND FOUND', 'Normal'), ('DAMAGED PROPERTY', 'High'), ('STOLEN ITEMS', 'Normal'), ('EMERGENCY INCIDENTS', 'High'), ('SEXUAL INCIDENTS', 'High')
add_incident_types | Incident types added: 5


In [10]:
def add_incident_locations(filepath):
    try: 
        ## establish connection
        db, cursor = establish_sql_connection()
        if db is None:
            print("add_incident_locations | Failed to establish a database connection.")
            return
        print(f'add_incident_locations | db established: {db.is_connected()}')

        ## read incident locations from CSV file
        df = pd.read_csv(filepath)
        unique_locations = df['Location'].unique()

        # Populate Incident_location_groups table with unique locations
        for location in unique_locations:
            try:
                query = f"INSERT INTO Incident_location_groups(location_group) VALUES ('{location}')"
                cursor.execute(query)
                db.commit()
                print(f"add_incident_locations | Location Group added: {location}")
            except mysql.connector.Error as err:
                print(f"add_incident_locations | Error adding location group: {err}")

        # Map CSV columns to MySQL columns and insert data into Incident_location table
        for index, row in df.iterrows():
            building = row['Buildings']
            location = row['Location']
            latitude = row['Latitude']
            longitude = row['Longitude']
            residence = row['Residence']

            # Get location_group_id from Incident_location_groups table
            query = f"SELECT id FROM Incident_location_groups WHERE location_group = '{location}'"
            cursor.execute(query)
            location_group_id = cursor.fetchone()[0]

            # Insert data into Incident_location table
            try:
                query = f"INSERT INTO Incident_location(name, location_group_id, latitude, longitude, is_residence) VALUES ('{building!r}', {location_group_id}, {latitude}, {longitude}, {residence})"
                cursor.execute(query)
                db.commit()
                print(f"add_incident_locations | Incident Location added: {building!r}")
            except mysql.connector.Error as err:
                print(f"add_incident_locations | Error adding incident location: {err}")

        cursor.close()
        db.close()

    except mysql.connector.Error as err:
        print(f"add_incident_locations | database error: {err}")

In [11]:
add_incident_locations("Incident_location.csv")

add_incident_locations | db established: True
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'KR' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'PGP' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'BTC' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'UCC' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'FASS' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'DNMS' for key 'Incident_location_groups.location_group'
add_incident_locations | Error adding location group: 1062 (23000): Duplicate entry 'FOS' for key 'Incident_locat

In [36]:

def add_incidents_from_csv(filepath):

    try: 
        ## establish connection
        db, cursor = establish_sql_connection()
        if db is None:
            print("add_incidents_from_csv | Failed to establish a database connection.")
            return
        print(f'add_incidents_from_csv | db established: {db.is_connected()}')

        ## read unique Incidents
        df = pd.read_csv(filepath)
        unique_incidents= df[['IncidentID','Description','Building','Incidents']].drop_duplicates()
        print(unique_incidents.shape)
        unique_incidents = unique_incidents.rename(columns={
            'Building': 'Location',
            'Incidents': 'Incident Type'
        })

        # Populate Incident_location_groups table with unique Incidents
        for index, row in unique_incidents.iterrows():
            try:
                # id
                id = row['IncidentID']
                #description
                description = row['Description']
                #location_id
                location = row['Location']
                query = f"SELECT id FROM Incident_locations WHERE location = '{location}'"
                cursor.execute(query)
                location_id = cursor.fetchone()[0]
                #incident_type_id
                incident_type = row['Incident Type']
                query = f"SELECT id FROM Incident_types WHERE type = '{incident_type}'"
                cursor.execute(query)
                incident_type_id = cursor.fetchone()[0]
                query = f"INSERT INTO Incidents(id,description,location_id,incident_type_id) VALUES \
                    ('{id}','{description}','{location_id}','{incident_type_id}')"
                cursor.execute(query)
            except mysql.connector.Error as err:
                print(f"add_incidents_from_csv | Error adding unique incident {id} to Incidents: {err}")
            finally:
                print(f"add_incidents_from_csv | Unique incidents added: {cursor.rowcount}")
                db.commit()

        ## read logs
        df['Time'] = pd.to_datetime(df['Time'],format="%d/%m/%y %H:%M")
        incident_logs = df[['IncidentID','Status','Priority','Time','User']].drop_duplicates()
        for index, row in incident_logs.iterrows():
            try:
                id = row['IncidentID']
                status = row['Status']
                priority = row['Priority']
                time = row['Time']
                user = row['User']
                query = f"SELECT id FROM Users WHERE username = '{user}'"
                cursor.execute(query)
                user_id = cursor.fetchone()[0]
                notes = ""

                ## Insert
                query = f"INSERT INTO Incident_logs(incident_id,status, priority,time,user_id,notes) VALUES \
                    ('{id}','{status}','{priority}','{time}','{user_id}','{notes}')"
                cursor.execute(query)
            except mysql.connector.Error as err:
                print(f"add_incidents_from_csv | Error adding log of incident {id} at {time} to Incidents: {err}")
        
        print(f"add_incidents_from_csv | Incident logs added: {index}")
        db.commit()

        cursor.close()
        db.close()

    except mysql.connector.Error as err:
        print(f"add_incident_locations | database error: {err}")

In [4]:
import os
print(os.getcwd())
add_incidents_from_csv("../../data/data_v0.4(back).csv")


/Users/chloetan/Desktop/DSA3101/proj/back/database/populate


NameError: name 'add_incidents_from_csv' is not defined

In [ ]:
import os
import pd
print(os.getcwd())
add_incidents_from_csv("../../data/data_v0.4(back).csv")

